### Методы валидации данных: кросс-валидация

### K-Fold

Этот метод случайным образом разбивает данные на $k$ непересекающихся блоков примерно одинакового размера. Поочередно каждый блок рассматривается как валидационная выборка, а остальные $k-1$ блоков — как обучающая выборка. Модель обучается на $k-1$ блоках и прогнозирует валидационный блок. Прогноз модели оценивается с помощью выбранного показателя: правильность (accuracy), среднеквадратическое отклонение (СКО, RMSE) и т.п. Процесс повторяется k раз, и мы получаем $k$ оценок, для которых рассчитывается среднее значение, являющееся итоговой оценкой модели.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

url = "https://raw.githubusercontent.com/maykulkarni/Machine-Learning-Notebooks/master/05.%20Model%20Evaluation/Social_Network_Ads.csv"
df = pd.read_csv(url)
X = df.iloc[:, 2:4] 
y = df.iloc[:, 4]

df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [2]:
X

,Age,EstimatedSalary
0,19,19000
1,35,20000
2,26,43000
3,27,57000
4,19,76000
...,...,...
395,46,41000
396,51,23000
397,50,20000
398,36,33000


In [3]:
y

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

StandardScaler масштабирует данные до единичной дисперсии. 

In [4]:
# Scale
from sklearn.preprocessing import StandardScaler
X_sca = StandardScaler()
X = X_sca.fit_transform(X)
print(X)

[[-1.78179743 -1.49004624]
 [-0.25358736 -1.46068138]
 [-1.11320552 -0.78528968]
 [-1.01769239 -0.37418169]
 [-1.78179743  0.18375059]
 [-1.01769239 -0.34481683]
 [-1.01769239  0.41866944]
 [-0.54012675  2.35674998]
 [-1.20871865 -1.07893824]
 [-0.25358736 -0.13926283]
 [-1.11320552  0.30121002]
 [-1.11320552 -0.52100597]
 [-1.6862843   0.47739916]
 [-0.54012675 -1.51941109]
 [-1.87731056  0.35993973]
 [-0.82666613  0.30121002]
 [ 0.89257019 -1.3138571 ]
 [ 0.70154394 -1.28449224]
 [ 0.79705706 -1.22576253]
 [ 0.98808332 -1.19639767]
 [ 0.70154394 -1.40195167]
 [ 0.89257019 -0.60910054]
 [ 0.98808332 -0.84401939]
 [ 0.70154394 -1.40195167]
 [ 0.79705706 -1.37258681]
 [ 0.89257019 -1.46068138]
 [ 1.08359645 -1.22576253]
 [ 0.89257019 -1.16703281]
 [-0.82666613 -0.78528968]
 [-0.63563988 -1.51941109]
 [-0.63563988  0.12502088]
 [-1.01769239  1.97500684]
 [-1.59077117 -1.5781408 ]
 [-0.92217926 -0.75592482]
 [-1.01769239  0.59485858]
 [-0.25358736 -1.25512738]
 [-0.44461362 -1.22576253]
 

In [12]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


kfold_cv = KFold(n_splits=5)
correct = 0
total = 0
for train_indices, test_indices in kfold_cv.split(X):
    X_train, X_test, y_train, y_test = X[train_indices], X[test_indices], \
                                        y[train_indices], y[test_indices]
    clf = SVC(kernel='linear', random_state=0).fit(X_train, y_train)
    correct += accuracy_score(y_test, clf.predict(X_test))
    total += 1
print("Accuracy: {0:.2f}".format(correct/total))

Accuracy: 0.82


In [26]:
clf = SVC(kernel='linear', random_state=0).fit(X_train, y_train)

In [27]:
# applying k-fold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(clf, X_train, y_train, cv=10)
print(accuracies, accuracies.mean(), accuracies.std())

[0.72222222 0.69444444 0.94444444 0.94444444 0.97222222 0.94444444
 0.83333333 0.75       0.80555556 0.91666667] 0.8527777777777776 0.0994196120454071


### LOO (Leave One Out)

Из $k$ объектов выборки обучение поочередно происходит на $k-1$ объекте, а тестируется оставшийся объект

***Вопрос***: При каком значении $k$ метод K-Fold эквивалентен LOO? 

In [35]:
from sklearn.model_selection import LeaveOneOut

loo_cv = LeaveOneOut()
correct = 0
total = 0
for train_indices, test_indices in loo_cv.split(X):
#     print("Train Indices: {}...".format(train_indices[:4]))
#     print("Test Indices: {}...".format(test_indices[:4]))
    X_train, X_test, y_train, y_test = X[train_indices], X[test_indices], \
                                        y[train_indices], y[test_indices]
    clf = SVC(kernel='linear', random_state=0).fit(X_train, y_train)
    correct += accuracy_score(y_test, clf.predict(X_test))
    total += 1
print("Accuracy: {0:.2f}".format(correct/total))

Accuracy: 0.84


### Stratified KFold

***Вопрос***: Подумайте, в каких случаях KFold может дать неправильное представление о результатах классификации?

Стратифицированное разделение сохранит статистику в исходном наборе: то есть, если исходный набор данных имеет 50% положительных и 50% отрицательных "ответов", то обучающий набор также будет иметь 50% положительных и 50% отрицательных "ответов".

In [39]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10)

correct = 0
total = 0
for train_indices, test_indices in skf.split(X, y):
#     print("Train Indices: {}...".format(train_indices[:4]))
#     print("Test Indices: {}...".format(test_indices[:4]))
    X_train, X_test, y_train, y_test = X[train_indices], X[test_indices], \
                                        y[train_indices], y[test_indices]
    clf = SVC(kernel='linear', random_state=0).fit(X_train, y_train)
    correct += accuracy_score(y_test, clf.predict(X_test))
    total += 1
print("Accuracy: {0:.2f}".format(correct/total))

Accuracy: 0.82


### TimeSeriesSplit

Полезно только в случае временных признаков и прогноза по временным рядам (например, цены на акции компаний).

In [40]:
from sklearn.model_selection import TimeSeriesSplit

X = np.random.rand(10, 2)
y = np.random.rand(10)
print(X)
print(y)

[[0.63842056 0.51422037]
 [0.47527633 0.77684345]
 [0.27466697 0.77778076]
 [0.264509   0.31006254]
 [0.63344775 0.98070725]
 [0.37706307 0.18881544]
 [0.35012962 0.81897467]
 [0.43657062 0.80620154]
 [0.36014178 0.24299861]
 [0.54076684 0.97521284]]
[0.02630708 0.78839643 0.05490618 0.04216088 0.85973401 0.86027591
 0.97026441 0.07871891 0.9988354  0.04523729]


In [41]:
tss = TimeSeriesSplit(n_splits=7)

for train_indices, test_indices in tss.split(X):
    print("Train indices: {0} Test indices: {1}".format(train_indices, test_indices))

Train indices: [0 1 2] Test indices: [3]
Train indices: [0 1 2 3] Test indices: [4]
Train indices: [0 1 2 3 4] Test indices: [5]
Train indices: [0 1 2 3 4 5] Test indices: [6]
Train indices: [0 1 2 3 4 5 6] Test indices: [7]
Train indices: [0 1 2 3 4 5 6 7] Test indices: [8]
Train indices: [0 1 2 3 4 5 6 7 8] Test indices: [9]
